import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split

# Example dictionary (patient data stored as Pandas DataFrames)
data = {
    "patient_1": pd.DataFrame({
        'x1': [1.0, 2.0, 3.0],
        'x2': [2.0, 3.0, 4.0],
        'x3': [1.0, 2.0, 1.5],
        'x4': [0.5, 0.7, 0.9],
        'y': [0, 1, 0]
    }),
    "patient_2": pd.DataFrame({
        'x1': [2.0, 3.0],
        'x2': [1.0, 2.0],
        'x3': [0.5, 0.7],
        'x4': [0.3, 0.6],
        'y': [1, 0]
    }),
    "patient_3": pd.DataFrame({
        'x1': [1.0, 1.0, 2.0, 2.0],
        'x2': [2.0, 3.0, 1.5, 3.0],
        'x3': [1.0, 0.8, 1.2, 1.1],
        'x4': [0.5, 0.7, 0.6, 0.8],
        'y': [0, 1, 0, 1]
    }),
    # Add more patients as needed
}

# Step 1: Split the data into training and test sets
patient_ids = list(data.keys())
train_ids, test_ids = train_test_split(patient_ids, test_size=0.2, random_state=42)

train_data = {k: data[k] for k in train_ids}
test_data = {k: data[k] for k in test_ids}

# Step 2: Prepare the data for training
def prepare_data(data_dict):
    sequences = []
    labels = []
    lengths = []

    for patient_id, df in data_dict.items():
        # Extract explanatory variables (x1, x2, x3, x4)
        sequence = torch.tensor(df[['x1', 'x2', 'x3', 'x4']].values, dtype=torch.float32)

        # Extract the target variable (y)
        label = torch.tensor(df['y'].values[0], dtype=torch.float32)  # Assuming consistent label across series

        sequences.append(sequence)
        labels.append(label)
        lengths.append(len(sequence))

    # Convert labels to a tensor
    labels = torch.tensor(labels, dtype=torch.float32)

    # Pad sequences to the maximum length
    padded_sequences = nn.utils.rnn.pad_sequence(sequences, batch_first=True)

    # Convert lengths to a tensor
    lengths = torch.tensor(lengths)

    return padded_sequences, labels, lengths

# Prepare training and test data
train_sequences, train_labels, train_lengths = prepare_data(train_data)
test_sequences, test_labels, test_lengths = prepare_data(test_data)

# Step 3: Define the GRU model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, lengths):
        packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.gru(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        idx = (lengths - 1).view(-1, 1).expand(len(lengths), output.size(2)).unsqueeze(1)
        last_output = output.gather(1, idx).squeeze(1)

        return torch.sigmoid(self.fc(last_output))

# Instantiate the model
input_size = train_sequences.size(2)  # Number of features (4 in this case: x1, x2, x3, x4)
hidden_size = 64  # Number of GRU units
model = GRUModel(input_size, hidden_size)

# Step 4: Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 5: Training Loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass for training
    train_outputs = model(train_sequences, train_lengths)
    loss = criterion(train_outputs.squeeze(), train_labels)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Step 6: Test the model (optional)
model.eval()
with torch.no_grad():
    test_outputs = model(test_sequences, test_lengths)
    predictions = (test_outputs.squeeze() > 0.5).float()
    print(f'Predictions: {predictions}')


In [5]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.model_selection import train_test_split
import torch.optim as optim
import pandas as pd
import pickle

In [4]:
# Read DataFrames from dictionary

# Load the dictionary from the file
path_to_read = '/Users/luisescobar/Documents/Thesis/DataSets/Dictionary'
#path_to_save = '/Users/luisescobar/Documents/Thesis/DataSets/Dictionary'
file_name = 'completo1007_(edit)_clotting.pkl'
name_to_read = f'{path_to_read}/{file_name}'

with open(name_to_read, 'rb') as file:
    loaded_dict = pickle.load(file)

In [8]:
# Split data into train and test
'''
Maybe will be a better idea to split the final DataFrames into clotting and no clotting. Once we have this split we
can divide in a better ratio for train and test. Plot a graph in the previous notebook to see the balance between classes   
'''
# Step 1: Split the data into training and test sets

# Extract the keys
keys = list(loaded_dict.keys())

# Split the keys (80% train, 20% test)
train_keys, test_keys = train_test_split(keys, test_size=0.2, random_state=42)

# Create the train and test dictionaries
train_dict = {key: loaded_dict[key] for key in train_keys}
test_dict = {key: loaded_dict[key] for key in test_keys}

print("Train Dictionary:", len(train_dict))
print("Test Dictionary:", len(test_dict))

Train Dictionary: 148
Test Dictionary: 37


In [ ]:
'''
columns=["Date__Heure","P_Access","P_Filter","P_Effluent","P_Return","Q_Blood_Pump",
          "Q_Replacement", "Q_Dialysate", "Q_PBP", "Q_Patient_Fluid_Removal", "DeltaP", "TMP", "TMPa", "trt", 
         "Patient_weight__Kg_", "Set"]

For training we could remove
0   Date__Heure ---> maybe we could find seasonality with this variable. However, we must transform the data  
13  trt 
14  Patient_weight__Kg_ ---> we could include it for other models to see if it produce a prediction difference
15  Set 
'''
columns=["P_Access","P_Filter","P_Effluent","P_Return","Q_Blood_Pump",
         "Q_Replacement", "Q_Dialysate", "Q_PBP", "Q_Patient_Fluid_Removal", 
         "DeltaP", "TMP", "TMPa", "Condition_1", "Condition_2", "Delta_P_ref", 
         "TMP_ref"]

In [ ]:
# Step 2: Prepare the data for training
def prepare_data(data_dict):
    sequences = []
    labels = []
    lengths = []

    for patient_id, df in data_dict.items():
        # Extract explanatory variables (x1, x2, x3, x4)
        sequence = torch.tensor(df[['x1', 'x2', 'x3', 'x4']].values, dtype=torch.float32)

        # Extract the target variable (y)
        label = torch.tensor(df['y'].values[0], dtype=torch.float32)  # Assuming consistent label across series

        sequences.append(sequence)
        labels.append(label)
        lengths.append(len(sequence))

    # Convert labels to a tensor
    labels = torch.tensor(labels, dtype=torch.float32)

    # Pad sequences to the maximum length
    padded_sequences = nn.utils.rnn.pad_sequence(sequences, batch_first=True)

    # Convert lengths to a tensor
    lengths = torch.tensor(lengths)

    return padded_sequences, labels, lengths

In [ ]:
# Prepare training and test data
train_sequences, train_labels, train_lengths = prepare_data(train_data)
test_sequences, test_labels, test_lengths = prepare_data(test_data)

In [ ]:
# Step 3: Define the GRU model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, lengths):
        packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.gru(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        idx = (lengths - 1).view(-1, 1).expand(len(lengths), output.size(2)).unsqueeze(1)
        last_output = output.gather(1, idx).squeeze(1)

        return torch.sigmoid(self.fc(last_output))

In [ ]:
# Instantiate the model
input_size = train_sequences.size(2)  # Number of features (4 in this case: x1, x2, x3, x4)
hidden_size = 64  # Number of GRU units
model = GRUModel(input_size, hidden_size)

In [ ]:
# Step 4: Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [ ]:
# Step 5: Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass for training
    train_outputs = model(train_sequences, train_lengths)
    loss = criterion(train_outputs.squeeze(), train_labels)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Step 6: Test the model (optional)
model.eval()
with torch.no_grad():
    test_outputs = model(test_sequences, test_lengths)
    predictions = (test_outputs.squeeze() > 0.5).float()
    print(f'Predictions: {predictions}')

Problems to address
How dictionary will work during Step 1
Should we normalize the values?
Double check the step # Extract the target variable (y) since is assuming that the same label for the entire series
What does this means?
